In [1]:
import os
import mne
import pandas as pd
import numpy as np

In [31]:
subj=4

subj_str = f'sub-{subj:03d}'
raw = mne.io.read_raw_fif(os.path.join('preprocessed_data/'+subj_str+'_pre.fif'), preload=True)




stimulus_dict = {
'Stimulus/hc/p/left': 4,
'Stimulus/hc/p/right': 5,
'Stimulus/hc/np/left': 6,
'Stimulus/hc/np/right': 7,
'Stimulus/lc/p/left': 8,
'Stimulus/lc/p/right': 9,
'Stimulus/lc/np/left': 10,
'Stimulus/lc/np/right': 11,
}

response_dict = {
'Response/car': 2,
'Response/face': 3,
}

stim_dict = {
    'Stimulus/car': 16,
    'Stimulus/face': 17,
}

data = pd.read_csv('results/beh_preprocessed.csv', sep='\t')
data = data[data['subj_idx']==subj_str]



# N170
events_from_annot, event_dict = mne.events_from_annotations(raw, event_id={
    'Stimulus/S 10': 4,
    'Stimulus/S 11': 5,
    'Stimulus/S 20': 6,
    'Stimulus/S 21': 7,
    'Stimulus/S 30': 8,
    'Stimulus/S 31': 9,
    'Stimulus/S 40': 10,
    'Stimulus/S 41': 11,})

if len(events_from_annot) != len(data):
    new_events = [1 if x == 4 or x == 5 else 2 if x == 6 or x == 7 else 3 if x == 8 or x == 9 else 4 for x in events_from_annot[:,2].tolist()]
    boo = np.zeros(len(data))

    
    diff_len = len(data) - len(new_events)
    new_events += [4] * diff_len
    
    boo = np.array(data['condition'] == new_events, dtype=np.int32)

    loc = []
    for i in range(len(boo)):
        boo = np.array(data['condition'] == new_events, dtype=np.int32)
        if not boo[i]:
            new_events[i+1:] = new_events[i:-1]
            loc.append(i)
            new_events[i] = data.iloc[i]['condition']
            boo[i] = 1 if data.iloc[i]['condition'] == new_events[i] else 0
    data.reset_index(drop=True, inplace=True)
    data_tmp = data.iloc[~data.index.isin(loc)]
else:
    data_tmp = data

for trial_num in range(data_tmp.shape[0]):
    if data_tmp.iloc[trial_num]['stim'] == 'car':
        events_from_annot[trial_num][2] = 16
    else:
        events_from_annot[trial_num][2] = 17

tmin = -1
tmax = 2
baseline = -0.1
epoch = mne.Epochs(raw, 
                events_from_annot, 
                event_id=stim_dict , 
                tmin=tmin, 
                tmax=tmax,
                baseline = (baseline,0), 
                preload=True, 
                picks=['eeg'])


data_tmp_face = data_tmp[data_tmp['stim']=='face']
epoch_face = epoch['Stimulus/face']

chs=['PO7', 'P7', 'P8','PO8']
n170_time_window = [0.13, 0.17]
n170_integ_window = [int(n170_time_window[0]*epoch.info['sfreq']), int(n170_time_window[1]*epoch.info['sfreq'])]
epoch_face_slice = epoch_face.pick_channels(chs).crop(tmin=n170_time_window[0], tmax=n170_time_window[1])
n170_amplitudes_range = np.mean(epoch_face_slice.get_data(),axis=1)

# N170 amplitude
n170_amplitudes = np.mean(n170_amplitudes_range,axis=1)
data_tmp.loc[data_tmp['stim']=='face', 'n170_amplitude'] = n170_amplitudes
data_tmp_n170_amplitudes = data_tmp.loc[data_tmp['stim']=='face', 'n170_amplitude']
data_tmp['n170_amplitude'] = data_tmp_n170_amplitudes
data['n170_amplitude'] = data_tmp_n170_amplitudes


# N170 slope
n170_slopes = np.apply_along_axis(lambda x: np.polyfit(range(len(x)), x, 1)[0], axis=1, arr=n170_amplitudes_range)
data_tmp.loc[data_tmp['stim']=='face', 'n170_slopes'] = n170_slopes
data_tmp_n170_slopes = data_tmp.loc[data_tmp['stim']=='face', 'n170_slopes']
data_tmp['n170_slopes'] = data_tmp_n170_slopes
data['n170_slopes'] = data_tmp_n170_slopes

# CPP baseline
tmin = -1
tmax = 2
baseline = -0.5
epoch = mne.Epochs(raw, 
                events_from_annot, 
                event_id=stim_dict , 
                tmin=tmin, 
                tmax=tmax,
                baseline = (baseline,0), 
                preload=True, 
                picks=['eeg'])
chs=['CPz','CP1','CP2']
baseline_time_window = [-0.5, 0]
epoch_slice = epoch.pick_channels(chs).crop(tmin=baseline_time_window[0], tmax=baseline_time_window[1])
baseline_range = np.mean(np.mean(epoch_slice.get_data(),axis=1),axis=1)
data_tmp['baseline'] = baseline_range
data['baseline'] = data_tmp['baseline']

# cpp 
events_from_annot, event_dict = mne.events_from_annotations(raw, event_id={'Stimulus/S  5': 2,
'Stimulus/S  6': 3,})




tmin = -1
tmax = 2
epoch = mne.Epochs(raw, 
                events_from_annot, 
                event_id=response_dict , 
                tmin=tmin, 
                tmax=tmax,
                baseline = None,
                preload=True, 
                picks=['eeg'])


chs=['CPz','CP1','CP2']
cpp_time_window = [-0.18, -0.08]
epoch_slice = epoch.pick_channels(chs).crop(tmin=cpp_time_window[0], tmax=cpp_time_window[1])
cpp_amplitudes_range = np.mean(epoch_slice.get_data(),axis=1)

if len(data)!= len(events_from_annot):
    new_events = events_from_annot[:,2].tolist()
    data.loc[data['key_press']=='car','key_press'] = 2
    data.loc[data['key_press']=='face','key_press'] = 3

    boo = np.zeros(len(data))

    # 找出长度不同的部分，并使用默认值 4 来填充 new_events 中的空缺
    diff_len = len(data) - len(new_events)
    new_events += [0] * diff_len

    # 根据条件判断条件是否满足，并将结果存储在 boo 中
    boo = np.array(data['key_press'] == new_events, dtype=np.int32)

    loc = []
    # 如果条件不满足，将 new_events 中的值顺延，使其与 data['condition'] 匹配
    for i in range(len(boo)):
        boo = np.array(data['key_press'] == new_events, dtype=np.int32)
        if not boo[i]:
            new_events[i+1:] = new_events[i:-1]
            loc.append(i)
            new_events[i] = data.iloc[i]['key_press']
            boo[i] = 1 if data.iloc[i]['key_press'] == new_events[i] else 0
    data.reset_index(drop=True, inplace=True)
    data_tmp = data.iloc[~data.index.isin(loc)]
    data.loc[data['key_press']==2,'key_press'] = 'car'
    data.loc[data['key_press']==3,'key_press'] = 'face' 
else:
    pass   

cpp_amplitudes = np.mean(cpp_amplitudes_range,axis=1)
data_tmp['cpp_amplitude'] = cpp_amplitudes
data['cpp_amplitude'] = data_tmp['cpp_amplitude']
data['cpp_amplitude'] = data['cpp_amplitude'] - data['baseline']

# cpp slope
cpp_slopes = np.apply_along_axis(lambda x: np.polyfit(range(len(x)), x, 1)[0], axis=1, arr=cpp_amplitudes_range)
data_tmp['cpp_slopes'] = cpp_slopes
data['cpp_slopes'] = data_tmp['cpp_slopes']

# cpp variance
cpp_var = np.var(cpp_amplitudes_range,axis=1)
data_tmp['cpp_var'] = cpp_var
data['cpp_var'] = data_tmp['cpp_var']

# cpp peak
cpp_time_window = [-0.2, 0.1]
epoch_slice = epoch.pick_channels(chs).crop(tmin=cpp_time_window[0], tmax=cpp_time_window[1])
cpp_amplitudes_range = np.mean(epoch_slice.get_data(),axis=1)
cpp_peak = np.max(cpp_amplitudes_range,axis=1) 
data_tmp['cpp_peak'] = cpp_peak
data['cpp_peak'] = data_tmp['cpp_peak']
data['cpp_peak'] = data['cpp_peak'] - data['baseline']
# cpp peak latency
cpp_peak_latency = np.argmax(cpp_amplitudes_range,axis=1)/epoch.info['sfreq']+cpp_time_window[0] +data_tmp.rt
data_tmp['cpp_peak_latency'] = cpp_peak_latency
data['cpp_peak_latency'] = data_tmp['cpp_peak_latency']


Opening raw data file preprocessed_data/sub-004_pre.fif...
    Range : 0 ... 610426 =      0.000 ...  1192.238 secs
Ready.
Reading 0 ... 610426  =      0.000 ...  1192.238 secs...


C:\Users\liuyikang\AppData\Local\Temp\ipykernel_13272\566060533.py:4: RuntimeWarning: This filename (preprocessed_data/sub-004_pre.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw = mne.io.read_raw_fif(os.path.join('preprocessed_data/'+subj_str+'_pre.fif'), preload=True)


Used Annotations descriptions: ['Stimulus/S 10', 'Stimulus/S 11', 'Stimulus/S 20', 'Stimulus/S 21', 'Stimulus/S 30', 'Stimulus/S 31', 'Stimulus/S 40', 'Stimulus/S 41']
Not setting metadata
282 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 282 events and 1537 original time points ...
0 bad epochs dropped
Not setting metadata
282 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 282 events and 1537 original time points ...


C:\Users\liuyikang\AppData\Local\Temp\ipykernel_13272\566060533.py:99: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_tmp.loc[data_tmp['stim']=='face', 'n170_amplitude'] = n170_amplitudes
C:\Users\liuyikang\AppData\Local\Temp\ipykernel_13272\566060533.py:101: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_tmp['n170_amplitude'] = data_tmp_n170_amplitudes
C:\Users\liuyikang\AppData\Local\Temp\ipykernel_13272\566060533.py:107: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fr

0 bad epochs dropped
Used Annotations descriptions: ['Stimulus/S  5', 'Stimulus/S  6']
Not setting metadata
283 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 283 events and 1537 original time points ...
0 bad epochs dropped


C:\Users\liuyikang\AppData\Local\Temp\ipykernel_13272\566060533.py:128: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_tmp['baseline'] = baseline_range
C:\Users\liuyikang\AppData\Local\Temp\ipykernel_13272\566060533.py:186: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_tmp['cpp_amplitude'] = cpp_amplitudes
C:\Users\liuyikang\AppData\Local\Temp\ipykernel_13272\566060533.py:192: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_

In [36]:
data = pd.read_csv('results/data_beh_eeg.csv',sep='\t')

,condition,coherence,prioritized,subj_idx,rt,key_press,correct,run,stim,n170_amplitude,n170_slopes,baseline,cpp_amplitude,cpp_slopes,cpp_var,cpp_peak,cpp_peak_latency
0,1.0,high,yes,sub-001,0.529206,face,1.0,1,face,-0.000006,2.340534e-07,-7.629888e-22,3.117677e-06,2.529905e-07,1.560474e-11,8.673606e-06,0.417096
1,2.0,high,no,sub-001,0.519501,car,1.0,1,car,NaN,NaN,9.063582e-24,4.987548e-06,1.093017e-08,7.937925e-12,1.023172e-05,0.346845
2,2.0,high,no,sub-001,0.611204,face,1.0,1,face,0.000002,-9.800590e-07,-5.932526e-23,1.536745e-06,6.506849e-08,8.048211e-12,7.147239e-06,0.489329
3,1.0,high,yes,sub-001,0.465089,car,1.0,1,car,NaN,NaN,2.175260e-22,-4.935784e-06,8.507541e-08,2.988462e-12,-2.095967e-06,0.360793
4,1.0,high,yes,sub-001,0.367166,car,1.0,1,car,NaN,NaN,-6.591696e-23,-3.300782e-06,1.873482e-07,1.284015e-11,7.866549e-08,0.262869
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4603,3.0,low,yes,sub-017,0.309091,car,1.0,2,car,NaN,NaN,4.268123e-22,1.327897e-06,4.849044e-08,7.588991e-12,5.365667e-06,0.161825
4604,3.0,low,yes,sub-017,0.261170,car,1.0,2,car,NaN,NaN,-2.389490e-23,3.138467e-06,2.693548e-08,9.850273e-13,4.508958e-06,0.088514
4605,3.0,low,yes,sub-017,0.271483,car,0.0,2,face,-0.000009,1.896555e-07,5.586462e-22,5.418563e-07,2.337735e-08,7.014867e-12,4.151314e-06,0.092968
4606,4.0,low,no,sub-017,0.283706,car,1.0,2,car,NaN,NaN,-1.977509e-22,1.007300e-06,1.866158e-08,1.355688e-12,3.289002e-06,0.154019


In [124]:
subj =11
ntimes = 1

subj_str = f'sub-{subj:03d}'
raw = mne.io.read_raw_fif(os.path.join('preprocessed_data/'+subj_str+'_pre.fif'), preload=True)




stimulus_dict = {
'Stimulus/hc/p/left': 4,
'Stimulus/hc/p/right': 5,
'Stimulus/hc/np/left': 6,
'Stimulus/hc/np/right': 7,
'Stimulus/lc/p/left': 8,
'Stimulus/lc/p/right': 9,
'Stimulus/lc/np/left': 10,
'Stimulus/lc/np/right': 11,
}

response_dict = {
'Response/car': 2,
'Response/face': 3,
}

stim_dict = {
    'Stimulus/car': 16,
    'Stimulus/face': 17,
}

data = pd.read_csv('results/beh_preprocessed.csv', sep='\t')
data = data[data['subj_idx']==subj_str]



# N170
events_from_annot, event_dict = mne.events_from_annotations(raw, event_id={
    'Stimulus/S 10': 4,
    'Stimulus/S 11': 5,
    'Stimulus/S 20': 6,
    'Stimulus/S 21': 7,
    'Stimulus/S 30': 8,
    'Stimulus/S 31': 9,
    'Stimulus/S 40': 10,
    'Stimulus/S 41': 11,})


if len(events_from_annot) != len(data):
    new_events = [1 if x == 4 or x == 5 else 2 if x == 6 or x == 7 else 3 if x == 8 or x == 9 else 4 for x in events_from_annot[:,2].tolist()]
    boo = np.zeros(len(data))

    
    diff_len = len(data) - len(new_events)
    new_events += [4] * diff_len
    
    boo = np.array(data['condition'] == new_events, dtype=np.int32)

    loc = []
    for i in range(len(boo)):
        boo = np.array(data['condition'] == new_events, dtype=np.int32)
        if not boo[i]:
            new_events[i+1:] = new_events[i:-1]
            loc.append(i)
            new_events[i] = data.iloc[i]['condition']
            boo[i] = 1 if data.iloc[i]['condition'] == new_events[i] else 0
    data.reset_index(drop=True, inplace=True)
    data_tmp = data.iloc[~data.index.isin(loc)]
else:
    data_tmp = data

for trial_num in range(data_tmp.shape[0]):
    if data_tmp.iloc[trial_num]['stim'] == 'car':
        events_from_annot[trial_num][2] = 16
    else:
        events_from_annot[trial_num][2] = 17

tmin = -1
tmax = 2.5
baseline = -0.1
epoch = mne.Epochs(raw, 
                events_from_annot, 
                event_id=stim_dict , 
                tmin=tmin, 
                tmax=tmax,
                baseline = (baseline,0), 
                preload=True, 
                picks=['eeg'])
# permutation
data_n170_var = np.zeros(ntimes)
data_cpp_var = np.zeros(ntimes)
for i in range(ntimes):
    index_perm = np.random.permutation(data_tmp.index)
    data_perm = data_tmp.loc[index_perm].reset_index(drop=True)
    rt = data_perm['rt'].values

    chs=['PO7', 'P7', 'P8','PO8']
    time_window = np.array([[-0.18, -0.08]])+ rt.reshape(-1, 1)+ 0.2
    n_samples = int((0.1)*raw.info['sfreq'])
    epoch_slice = np.zeros((len(epoch), len(chs), n_samples))
    for j in range(len(epoch)):
        if pd.isna(data_perm['rt'][j]):
            epoch_slice[j] = epoch[j].pick_channels(chs).crop(tmin=time_window[j][0], tmax=time_window[j][1]).get_data()[:, :, :n_samples]*0
        else:
            epoch_slice[j] = epoch[j].pick_channels(chs).crop(tmin=time_window[j][0], tmax=time_window[j][1]).get_data()[:, :, :n_samples]

    n170_var_range = np.mean(np.mean(epoch_slice,axis=1),axis=1)
    n170_var_range = n170_var_range[np.where(n170_var_range!=0)]
    n170_var = np.var(n170_var_range)
    data_n170_var[i] = n170_var


    chs=['CPz', 'CP1', 'CP2']
    time_window = np.array([[-0.18, -0.08]])+ rt.reshape(-1, 1)+ 0.2
    epoch_slice = np.zeros((len(epoch), len(chs), n_samples))
    for j in range(len(epoch)):
        if pd.isna(data_perm['rt'][j]):
            epoch_slice[j] = epoch[j].pick_channels(chs).crop(tmin=time_window[j][0], tmax=time_window[j][1]).get_data()[:, :, :n_samples]*0
        else:
            epoch_slice[j] = epoch[j].pick_channels(chs).crop(tmin=time_window[j][0], tmax=time_window[j][1]).get_data()[:, :, :n_samples]

    cpp_var_range = np.mean(np.mean(epoch_slice,axis=1),axis=1)
    cpp_var_range = cpp_var_range[np.where(cpp_var_range!=0)]
    cpp_var = np.var(cpp_var_range)
    data_cpp_var[i] = cpp_var

    x1 = pd.DataFrame({str(subj):data_n170_var})
    x2 = pd.DataFrame({str(subj):data_cpp_var})

Opening raw data file preprocessed_data/sub-011_pre.fif...
    Range : 0 ... 646000 =      0.000 ...  1261.719 secs
Ready.
Reading 0 ... 646000  =      0.000 ...  1261.719 secs...


C:\Users\liuyikang\AppData\Local\Temp\ipykernel_13272\72797581.py:5: RuntimeWarning: This filename (preprocessed_data/sub-011_pre.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw = mne.io.read_raw_fif(os.path.join('preprocessed_data/'+subj_str+'_pre.fif'), preload=True)


Used Annotations descriptions: ['Stimulus/S 10', 'Stimulus/S 11', 'Stimulus/S 20', 'Stimulus/S 21', 'Stimulus/S 30', 'Stimulus/S 31', 'Stimulus/S 40', 'Stimulus/S 41']
Not setting metadata
288 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 288 events and 1793 original time points ...
0 bad epochs dropped


In [77]:
n170_vars=pd.DataFrame()
cpp_vars=pd.DataFrame()
for i in range(17):
    if i+1 != 2:
        n170_var = permutate_var(subj=i+1, ntimes=1)[0]        
        cpp_var = permutate_var(subj=i+1, ntimes=1)[1]
        n170_vars = pd.concat([n170_vars,n170_var],axis=1).reset_index(drop=True)
        cpp_vars = pd.concat([cpp_vars,cpp_var],axis=1).reset_index(drop=True)
        n170_vars.to_csv('results/n170_vars.csv', sep='\t', index=False)
        cpp_vars.to_csv('results/cpp_vars.csv', sep='\t', index=False)
    else:
        pass

Opening raw data file preprocessed_data/sub-001_pre.fif...


C:\Users\liuyikang\AppData\Local\Temp\ipykernel_13272\700895266.py:4: RuntimeWarning: This filename (preprocessed_data/sub-001_pre.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw = mne.io.read_raw_fif(os.path.join('preprocessed_data/'+subj_str+'_pre.fif'), preload=True)


    Range : 0 ... 548320 =      0.000 ...  1070.938 secs
Ready.
Reading 0 ... 548320  =      0.000 ...  1070.938 secs...
Used Annotations descriptions: ['Stimulus/S 10', 'Stimulus/S 11', 'Stimulus/S 20', 'Stimulus/S 21', 'Stimulus/S 30', 'Stimulus/S 31', 'Stimulus/S 40', 'Stimulus/S 41']
Not setting metadata
288 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 288 events and 1793 original time points ...
0 bad epochs dropped
Opening raw data file preprocessed_data/sub-001_pre.fif...
    Range : 0 ... 548320 =      0.000 ...  1070.938 secs
Ready.
Reading 0 ... 548320  =      0.000 ...  1070.938 secs...


C:\Users\liuyikang\AppData\Local\Temp\ipykernel_13272\700895266.py:4: RuntimeWarning: This filename (preprocessed_data/sub-001_pre.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw = mne.io.read_raw_fif(os.path.join('preprocessed_data/'+subj_str+'_pre.fif'), preload=True)


Used Annotations descriptions: ['Stimulus/S 10', 'Stimulus/S 11', 'Stimulus/S 20', 'Stimulus/S 21', 'Stimulus/S 30', 'Stimulus/S 31', 'Stimulus/S 40', 'Stimulus/S 41']
Not setting metadata
288 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 288 events and 1793 original time points ...
0 bad epochs dropped
Opening raw data file preprocessed_data/sub-003_pre.fif...


...

Used Annotations descriptions: ['Stimulus/S 10', 'Stimulus/S 11', 'Stimulus/S 20', 'Stimulus/S 21', 'Stimulus/S 30', 'Stimulus/S 31', 'Stimulus/S 40', 'Stimulus/S 41']
Not setting metadata
288 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 288 events and 1793 original time points ...
0 bad epochs dropped
Opening raw data file preprocessed_data/sub-017_pre.fif...


C:\Users\liuyikang\AppData\Local\Temp\ipykernel_13272\700895266.py:4: RuntimeWarning: This filename (preprocessed_data/sub-017_pre.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw = mne.io.read_raw_fif(os.path.join('preprocessed_data/'+subj_str+'_pre.fif'), preload=True)


    Range : 0 ... 548300 =      0.000 ...  1070.898 secs
Ready.
Reading 0 ... 548300  =      0.000 ...  1070.898 secs...
Used Annotations descriptions: ['Stimulus/S 10', 'Stimulus/S 11', 'Stimulus/S 20', 'Stimulus/S 21', 'Stimulus/S 30', 'Stimulus/S 31', 'Stimulus/S 40', 'Stimulus/S 41']
Not setting metadata
288 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 288 events and 1793 original time points ...
0 bad epochs dropped
Opening raw data file preprocessed_data/sub-017_pre.fif...
    Range : 0 ... 548300 =      0.000 ...  1070.898 secs
Ready.
Reading 0 ... 548300  =      0.000 ...  1070.898 secs...


C:\Users\liuyikang\AppData\Local\Temp\ipykernel_13272\700895266.py:4: RuntimeWarning: This filename (preprocessed_data/sub-017_pre.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw = mne.io.read_raw_fif(os.path.join('preprocessed_data/'+subj_str+'_pre.fif'), preload=True)


Used Annotations descriptions: ['Stimulus/S 10', 'Stimulus/S 11', 'Stimulus/S 20', 'Stimulus/S 21', 'Stimulus/S 30', 'Stimulus/S 31', 'Stimulus/S 40', 'Stimulus/S 41']
Not setting metadata
288 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 288 events and 1793 original time points ...
0 bad epochs dropped


In [78]:
n170_vars

,1,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,1.486065e-11,1.720725e-11,2.754683e-11,1.482757e-11,2.305192e-11,7.916857e-11,2.318695e-11,1.964881e-11,2.859990e-11,2.498570e-11,1.781977e-11,1.230196e-11,1.976539e-11,1.996131e-11,4.376081e-11,1.062059e-10


In [79]:
cpp_vars

,1,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,1.298861e-11,8.283731e-12,2.266064e-11,1.447573e-11,6.850459e-12,2.407564e-11,1.203383e-11,1.183602e-11,1.955740e-11,2.010774e-11,2.237464e-11,9.137667e-12,9.909815e-12,8.990994e-12,3.380642e-11,6.400909e-11
